## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-17-01-00-17 +0000,latimes,He brandished a knife and demanded she hand ov...,https://www.latimes.com/california/story/2025-...
1,2025-09-17-01-00-00 +0000,wsj,America Loves Cocaine Again—Mexico’s New Drug ...,https://www.wsj.com/world/americas/mexico-drug...
2,2025-09-17-01-00-00 +0000,wsj,High-earners and older Americans are faring be...,https://www.wsj.com/economy/us-economy-analysi...
3,2025-09-17-01-00-00 +0000,wsj,This week’s Fed meeting is shaping up to be th...,https://www.wsj.com/economy/central-banking/fe...
4,2025-09-17-00-53-17 +0000,missionlocal,Live election results: Joel Engardio recall in...,https://missionlocal.org/2025/09/sf-district-4...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
82,trump,55
54,kirk,31
556,gaza,25
53,charlie,22
13,new,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
62,2025-09-16-21-45-00 +0000,wsj,In her first public comments since Trump fired...,https://www.wsj.com/economy/jobs/fired-bls-chi...,119
224,2025-09-16-14-44-00 +0000,wsj,FBI Director Kash Patel is defending his work ...,https://www.wsj.com/politics/policy/fbi-direct...,112
136,2025-09-16-18-31-12 +0000,bbc,Trump says he will sue New York Times for $15bn,https://www.bbc.com/news/articles/cevz419pk8vo...,111
315,2025-09-16-08-49-51 +0000,cbc,Trump launches $15B US defamation lawsuit agai...,https://www.cbc.ca/news/world/trump-lawsuit-ne...,109
40,2025-09-16-22-28-55 +0000,nypost,Hochul argues Zohran Mamdani would be best can...,https://nypost.com/2025/09/16/us-news/hochul-a...,106


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
62,119,2025-09-16-21-45-00 +0000,wsj,In her first public comments since Trump fired...,https://www.wsj.com/economy/jobs/fired-bls-chi...
224,105,2025-09-16-14-44-00 +0000,wsj,FBI Director Kash Patel is defending his work ...,https://www.wsj.com/politics/policy/fbi-direct...
155,101,2025-09-16-17-37-00 +0000,wsj,Israel Launches New Ground Offensive Seeking t...,https://www.wsj.com/world/middle-east/israel-l...
98,90,2025-09-16-20-14-00 +0000,wsj,Democrats are introducing a bill in the Senate...,https://www.wsj.com/economy/central-banking/se...
40,51,2025-09-16-22-28-55 +0000,nypost,Hochul argues Zohran Mamdani would be best can...,https://nypost.com/2025/09/16/us-news/hochul-a...
226,50,2025-09-16-14-43-00 +0000,wsj,Judge Throws Out Terror-Related Murder Charges...,https://www.wsj.com/us-news/law/judge-throws-o...
255,42,2025-09-16-13-19-35 +0000,nypost,DA Fani Willis loses bid to appeal Trump’s Geo...,https://nypost.com/2025/09/16/us-news/da-fani-...
240,40,2025-09-16-14-02-24 +0000,nypost,Russia launches large-scale attack on Ukrainia...,https://nypost.com/2025/09/16/world-news/russi...
32,38,2025-09-16-22-59-07 +0000,nypost,Trump’s TikTok deal could face hitch over bill...,https://nypost.com/2025/09/16/business/tiktok-...
150,36,2025-09-16-17-49-00 +0000,wsj,A federal appeals court on Monday night reject...,https://www.wsj.com/economy/central-banking/ap...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
